In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import time

url = 'https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query=%EB%AA%BD%ED%82%BD+%EC%9D%BC%EC%82%B0%EB%B0%A4%EB%A6%AC%EB%8B%A8%EA%B8%B8%EC%B9%B4%ED%8E%98+%EB%B3%B8%EC%A0%90'


In [15]:
import re
import time
import pandas as pd
from urllib.parse import quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup


def extract_korean(text: str) -> str:
    """문자열에서 한글만 추출해서 공백으로 이어붙임."""
    if not text:
        return ""
    result = re.compile('[가-힣]+').findall(text)
    return " ".join(result)


def build_search_url(store_name: str) -> str:
    """가게 이름으로 네이버 블로그 검색 URL 생성."""
    query = quote_plus(store_name)  # 공백, 한글 등 인코딩
    url = f"https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query={query}"
    return url


def crawl_naver_blog_reviews(store_name: str, scroll_count: int = 1) -> pd.DataFrame:
    """
    네이버 블로그에서 특정 가게 이름으로 검색한 뒤
    블로그 글 제목, 링크, 내용, 한글만 추출한 내용을 크롤링해서 DataFrame 반환 + CSV 저장.

    Parameters
    ----------
    store_name : str
        검색할 가게 이름 (예: '몽키 일산 밤리단길카페 본점')
    scroll_count : int
        검색 결과 화면에서 PAGE_DOWN 할 횟수 (기본 1번)

    Returns
    -------
    pd.DataFrame
        titles, links, contents, only_kor_contents 컬럼을 가진 DataFrame
    """

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    # 1. 크롬 드라이버 실행
    driver = webdriver.Chrome(options=chrome_options)

    try:
        # 2. 검색 페이지 접속
        search_url = build_search_url(store_name)
        driver.get(search_url)
        time.sleep(3)

        # 3. 검색 결과 페이지 스크롤 (조금 더 로딩되게)
        body = driver.find_element(By.CSS_SELECTOR, 'body')
        for _ in range(scroll_count):
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)

        # 4. HTML 가져와서 BeautifulSoup으로 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 5. 검색 결과에서 블로그 제목/링크 추출 (현재 네이버 구조 기준)
        #'div > div.sds-comps-vertical-layout.sds-comps-full-layout.N8JknjVu2Kc8aLavF6ZA > a'
        url_soup = soup.select('div > div.sds-comps-vertical-layout.sds-comps-full-layout.N8JknjVu2Kc8aLavF6ZA > a') + soup.select('div > div.sds-comps-vertical-layout.sds-comps-full-layout.MkfloTjOr2Rg4LLlLwVA > a')

        titles = []
        links = []

        for t in url_soup:
            title_text = t.get_text().strip()
            link = t.attrs.get('href', '').strip()
            if not link:
                continue

            print(title_text, link)  # 디버깅용 출력
            titles.append(title_text)
            links.append(link)

        # 6. 각 블로그 글에 들어가서 본문 크롤링
        contents = []

        for link in links:
            try:
                driver.get(link)
                time.sleep(2)

                # 새 에디터/구 에디터 섞여 있어서 try-except
                try:
                    # 구버전 블로그(iframe) 구조
                    driver.switch_to.frame("mainFrame")
                except Exception:
                    print('error')
                    # iframe이 없을 수도 있으니 무시
                    pass

                try:
                    text = driver.find_element(By.CSS_SELECTOR, 'div.se-main-container').text
                except Exception:
                    # 다른 구조의 블로그일 경우 대비
                    try:
                        text = driver.find_element(By.CSS_SELECTOR, 'div#postViewArea').text
                    except Exception:
                        text = ""

                contents.append(text)
            except Exception as e:
                print(f"[ERROR] {link} 크롤링 중 오류: {e}")
                contents.append("")

            # 항상 frame 다시 초기화
            driver.switch_to.default_content()

        # 7. 한글만 남긴 컬럼 생성
        only_kor_contents = [extract_korean(c) for c in contents]

        # 8. DataFrame 생성
        review_data = pd.DataFrame({
            'titles': titles,
            'links': links,
            'contents': contents,
            'only_kor_contents': only_kor_contents
        })

        # 9. 파일명에 쓸 수 없는 문자 제거 후 CSV 저장
        safe_store_name = re.sub(r'[\\/*?:"<>|]', "_", store_name)
        filename = f"{safe_store_name}_review_data.csv"
        review_data.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"✅ '{filename}' 파일로 저장 완료")

        return review_data

    finally:
        driver.quit()


def combine_review(review_list):
    input_string = ''
    for i in review_list:
        input_string += i
    return input_string


if __name__ == "__main__":
    # 예시 실행
    store = "카피로우 일산밤리단길카페점"
    df = crawl_naver_blog_reviews(store_name=store, scroll_count=1)
    input_string = combine_review(df['only_kor_contents'])


✅ '카피로우 일산밤리단길카페점_review_data.csv' 파일로 저장 완료


In [19]:
import re
import time
import pandas as pd
from urllib.parse import quote_plus

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup


def extract_korean(text: str) -> str:
    """문자열에서 한글만 추출해서 공백으로 이어붙임."""
    if not text:
        return ""
    result = re.compile('[가-힣]+').findall(text)
    return " ".join(result)


def build_search_url(store_name: str) -> str:
    """가게 이름으로 네이버 블로그 검색 URL 생성."""
    query = quote_plus(store_name)  # 공백, 한글 등 인코딩
    url = f"https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query={query}"
    return url


def _create_driver() -> webdriver.Chrome:
    """크롬 드라이버 생성 (속도 최적화 옵션 추가)."""
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    # 이미지 로딩 끄기 (네트워크/렌더링 부담 감소)
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(options=chrome_options)
    driver.set_window_size(1280, 800)
    return driver


def crawl_naver_blog_reviews(
    store_name: str,
    scroll_count: int = 1,
    max_posts: int = 5,
) -> pd.DataFrame:
    """
    네이버 블로그에서 특정 가게 이름으로 검색한 뒤
    블로그 글 제목, 링크, 내용, 한글만 추출한 내용을 크롤링해서 DataFrame 반환 + CSV 저장.

    Parameters
    ----------
    store_name : str
        검색할 가게 이름 (예: '몽키 일산 밤리단길카페 본점')
    scroll_count : int
        검색 결과 화면에서 PAGE_DOWN 할 횟수 (기본 1번)
    max_posts : int
        상위 몇 개 블로그 글만 크롤링할지 (기본 5개)

    Returns
    -------
    pd.DataFrame
        titles, links, contents, only_kor_contents 컬럼을 가진 DataFrame
    """

    driver = _create_driver()
    wait = WebDriverWait(driver, 10)

    try:
        # 1. 검색 페이지 접속
        search_url = build_search_url(store_name)
        driver.get(search_url)

        # 2. 검색 결과 페이지 로딩 완료까지 대기
        #    (블로그 카드 레이아웃이 뜰 때까지)
        try:
            wait.until(
                EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR,
                     "div.sds-comps-vertical-layout.sds-comps-full-layout")
                )
            )
        except Exception:
            # 그래도 못 찾으면 그냥 현재 페이지 기준으로 진행
            pass

        # 3. 스크롤 (조금만)
        body = driver.find_element(By.CSS_SELECTOR, "body")
        for _ in range(scroll_count):
            body.send_keys(Keys.PAGE_DOWN)
            # 너무 오래 기다릴 필요 없이 짧게
            time.sleep(0.5)

        # 4. HTML 가져와서 BeautifulSoup으로 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # 5. 검색 결과에서 블로그 제목/링크 추출
        url_soup = (
            soup.select(
                "div > div.sds-comps-vertical-layout.sds-comps-full-layout.N8JknjVu2Kc8aLavF6ZA > a"
            )
            + soup.select(
                "div > div.sds-comps-vertical-layout.sds-comps-full-layout.MkfloTjOr2Rg4LLlLwVA > a"
            )
            +soup.select(
                'div > div.sds-comps-vertical-layout.sds-comps-full-layout._IJWW1BBVoZsf1hqzdtq > a'
                )
        )
        print('url', url_soup)
        # 상위 max_posts개만 사용
        url_soup = url_soup[:max_posts]

        titles = []
        links = []

        for t in url_soup:
            title_text = t.get_text().strip()
            print(title_text)
            link = t.attrs.get("href", "").strip()
            if not link:
                continue

            print("[BLOG]", title_text, link)  # 디버깅용 출력
            titles.append(title_text)
            links.append(link)

        # 6. 각 블로그 글에 들어가서 본문 크롤링
        contents = []

        for idx, link in enumerate(links):
            try:
                driver.get(link)

                # 블로그 본문 로딩 기다리기
                # 새 에디터
                try:
                    wait.until(
                        EC.presence_of_element_located(
                            (By.CSS_SELECTOR, "iframe#mainFrame, div.se-main-container, div#postViewArea")
                        )
                    )
                except Exception:
                    pass

                # 구버전 블로그(iframe) 구조일 경우 frame 전환
                switched_to_frame = False
                try:
                    driver.switch_to.frame("mainFrame")
                    switched_to_frame = True
                except Exception:
                    # iframe이 없을 수도 있으니 무시
                    pass

                text = ""
                # 새 에디터
                try:
                    text = driver.find_element(
                        By.CSS_SELECTOR, "div.se-main-container"
                    ).text
                except Exception:
                    # 구 에디터
                    try:
                        text = driver.find_element(
                            By.CSS_SELECTOR, "div#postViewArea"
                        ).text
                    except Exception:
                        text = ""

                contents.append(text)

                # 항상 frame 다시 초기화
                if switched_to_frame:
                    driver.switch_to.default_content()

                print(f"[CONTENT] {idx+1}/{len(links)} 글 크롤링 완료")

            except Exception as e:
                print(f"[ERROR] {link} 크롤링 중 오류: {e}")
                contents.append("")
                try:
                    driver.switch_to.default_content()
                except Exception:
                    pass


        driver.get(search_url)
        time.sleep(3)

        print("page title:", driver.title)
        print("page length:", len(driver.page_source))

        # 7. 한글만 남긴 컬럼 생성
        only_kor_contents = [extract_korean(c) for c in contents]

        # 8. DataFrame 생성
        review_data = pd.DataFrame(
            {
                "titles": titles,
                "links": links,
                "contents": contents,
                "only_kor_contents": only_kor_contents,
            }
        )

        # 9. 파일명에 쓸 수 없는 문자 제거 후 CSV 저장
        safe_store_name = re.sub(r'[\\/*?:"<>|]', "_", store_name)
        filename = f"{safe_store_name}_review_data.csv"
        review_data.to_csv(filename, index=False, encoding="utf-8-sig")
        print(f"✅ '{filename}' 파일로 저장 완료")

        return review_data

    finally:
        driver.quit()


def combine_review(review_list):
    input_string = ""
    for i in review_list:
        input_string += i
    return input_string


def build_review_input_text(review_series, max_chars_per_review=2000):
    texts = []
    for raw in review_series:
        if not isinstance(raw, str):
            continue
        text = raw.strip()
        if not text:
            continue
        # 너무 긴 리뷰는 앞부분만 사용
        text = text[:max_chars_per_review]
        texts.append(text)

    # 리뷰들 사이에 구분선을 넣어서 합치기
    return "\n\n---\n\n".join(texts)


if __name__ == "__main__":
    # 예시 실행
    store = "뒷북서재"
    df = crawl_naver_blog_reviews(store_name=store, scroll_count=1, max_posts=5)
    input_string = build_review_input_text(df["only_kor_contents"])
    print(len(df), "개의 블로그 글 크롤링 완료")


url [<a class="fender-ui_228e3bd1 vnDTNtnoko3GR0aJilUy" data-heatmap-target=".nblg" href="https://blog.naver.com/arosa0206/223327567292" nocr="1" target="_blank"><span class="sds-comps-text sds-comps-text-ellipsis sds-comps-text-ellipsis-1 sds-comps-text-type-headline1 sds-comps-text-weight-sm">일산 밤리단길 힐링 북카페 <mark>뒷북서재</mark></span></a>, <a class="fender-ui_228e3bd1 vnDTNtnoko3GR0aJilUy" data-heatmap-target=".nblg" href="https://blog.naver.com/ageeable/223822881724" nocr="1" target="_blank"><span class="sds-comps-text sds-comps-text-ellipsis sds-comps-text-ellipsis-1 sds-comps-text-type-headline1 sds-comps-text-weight-sm">일산 정발산 조용한 북카페 추천 책과 만년필이 있는 <mark>뒷북서재</mark> 주차정보</span></a>, <a class="fender-ui_228e3bd1 vnDTNtnoko3GR0aJilUy" data-heatmap-target=".nblg" href="https://blog.naver.com/minin0702/224033987266" nocr="1" target="_blank"><span class="sds-comps-text sds-comps-text-ellipsis sds-comps-text-ellipsis-1 sds-comps-text-type-headline1 sds-comps-text-weight-sm"><mark>뒷북서재</ma

In [5]:
len(input_string)

7497

In [7]:
len(input_string)

6116

In [2]:
def combine_review(review_list):
    input_string = ''
    for i in review_list:
        input_string += i
    return input_string

In [ ]:
if __name__ == "__main__":
    # 예시 실행
    store = "카피로우 일산밤리단길카페점"
    df = crawl_naver_blog_reviews(store_name=store, scroll_count=1)
    input_string = combine_review(df['only_kor_contents'])

디저트가 특히 맛있었던 일산 카페, 카피로우 일산밤리단길카페점 방문기 https://blog.naver.com/hidew777/224076747828
카피로우 일산밤리단길카페점 솔직후기 https://blog.naver.com/rhdmsdl0720/223996105928
카피로우 일산밤리단길카페점 내돈내산 솔직후기 https://blog.naver.com/rkguscjswo26/223891656310
“카피로우” 일산밤리단길카페점.크렘당쥬?프랑스디저트 천사의크림일산 밤리단길카페추천 https://blog.naver.com/comma2024/223785598110
[경기/고양시] 일산동구 마두동 밤리단길 데이트 카페 추천 무화과 디저트 맛집 찾고 있다면? ‘카피로우 일산밤리단길카페점’ - 아메리카노, 무화과요거트케이크(글루텐프리) https://blog.naver.com/vivid_07/224064134098
일산 카페 찐맛집, 카피로우 일산밤리단길카페점 https://blog.naver.com/soheee666/223960234321
밤리단길 디저트 맛집 카피로우 느좋 테라스 카페 https://blog.naver.com/enterside/223918465299
일산 밤리단길 | 카피로우 https://blog.naver.com/rafig0913/223881827603
✅ '카피로우 일산밤리단길카페점_review_data.csv' 파일로 저장 완료


KeyError: 'only_kor_review'

In [5]:
input_string = combine_review(df['only_kor_contents'])

# Langchain

In [11]:
from typing import List
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
import os
from dotenv import load_dotenv

load_dotenv()

# --- 1) 구조화된 Output 모델 ---
class ReviewExtraction(BaseModel):
    main_menu: List[str] = Field(
        ...,
        description="가게에서 많이 언급되는 대표 메뉴 키워드들 (예: 소금빵, 아메리카노, 고구마라떼)"
    )
    atmosphere: List[str] = Field(
        ...,
        description="가게 분위기, 경험, 매장 특징 키워드들 (예: 아늑한, 감성적인, 좌석이 넓은)"
    )
    recommended_for: List[str] = Field(
        ...,
        description="어떤 유형의 사람이 방문하면 좋은지 (예: 연인과 함께, 친구와 수다, 반려견과 함께)"
    )


# --- 2) LLM + 구조화 출력 준비 ---
model = ChatOpenAI(
    model_name="gpt-5-nano",
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.2
).with_structured_output(ReviewExtraction)


# --- 3) 프롬프트 ---
prompt = PromptTemplate.from_template("""
너는 한국어 네이버 블로그 리뷰를 분석해서
가게의 대표 메뉴, 분위기, 추천 대상을 키워드로만 뽑는 역할을 한다.

아래 리뷰 텍스트를 보고,
각 항목당 3~4개의 핵심 키워드를 한국어로만 추출해라.

- main_menu: 자주 언급되는 메뉴 이름
- atmosphere: 매장의 분위기/경험/특징
- recommended_for: 어떤 사람이 방문하면 좋을지 (ex. 연인, 친구, 반려견과 함께 등)

반드시 키워드 위주의 짧은 표현만 사용해라.

리뷰 텍스트:
----------------
{text}
----------------
""")

# --- 4) LCEL 체인 ---
summarize_chain = prompt | model


# --- 5) 실행 예제 ---


def extract_review_keywords(input_text: str) -> ReviewExtraction:
    """
    한 가게의 리뷰 전체를 하나의 문자열로 받아서
    대표 메뉴 / 분위기 / 추천 대상을 추출한다.
    """
    if not input_text.strip():
        # 비어 있으면 그냥 기본값 리턴
        return ReviewExtraction(
            main_menu=[],
            atmosphere=[],
            recommended_for=[],
        )

    result: ReviewExtraction = summarize_chain.invoke({"text": input_text})
    return result
result = extract_review_keywords(input_string)
print(result)
print(result.main_menu)
print(result.atmosphere)
print(result.recommended_for)


main_menu=['카페라떼', '땅콩크림라떼', '단호박케이크', '무화과요거트케이크'] atmosphere=['따뜻하고 아늑한 분위기', '화이트톤 우드 인테리어', '조용하고 편안한 공간', '반려견 동반 가능'] recommended_for=['연인', '친구', '반려견과 함께 방문', '디저트/케이크 애호가']
['카페라떼', '땅콩크림라떼', '단호박케이크', '무화과요거트케이크']
['따뜻하고 아늑한 분위기', '화이트톤 우드 인테리어', '조용하고 편안한 공간', '반려견 동반 가능']
['연인', '친구', '반려견과 함께 방문', '디저트/케이크 애호가']


# 3개 가게 동시 리뷰 요약

In [2]:
# crawling.py
import re
import time
from urllib.parse import quote_plus

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def extract_korean(text: str) -> str:
    """문자열에서 한글만 추출해서 공백으로 이어붙임."""
    if not text:
        return ""
    result = re.compile("[가-힣]+").findall(text)
    return " ".join(result)


def build_search_url(store_name: str) -> str:
    """가게 이름으로 네이버 블로그 검색 URL 생성."""
    query = quote_plus(store_name)
    url = f"https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query={query}"
    return url


def _create_driver() -> webdriver.Chrome:
    """크롬 드라이버 생성 (속도 최적화 옵션 포함)."""
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")

    # 이미지 로딩 끄기 (속도 + 트래픽 감소)
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(options=chrome_options)
    driver.set_window_size(1280, 800)
    return driver


def crawl_naver_blog_reviews(
    store_name: str,
    scroll_count: int = 1,
    max_posts: int = 5,
) -> pd.DataFrame:
    """
    네이버 블로그에서 특정 가게 이름으로 검색한 뒤
    블로그 글 제목, 링크, 내용, 한글만 추출한 내용을 크롤링해서 DataFrame 반환.

    컬럼: titles, links, contents, only_kor_contents
    """

    driver = _create_driver()
    wait = WebDriverWait(driver, 10)

    try:
        # 1. 검색 페이지 접속
        search_url = build_search_url(store_name)
        driver.get(search_url)

        # 2. 검색 결과 로딩 대기
        try:
            wait.until(
                EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, "div.sds-comps-vertical-layout.sds-comps-full-layout")
                )
            )
        except Exception:
            pass

        # 3. 스크롤 (조금만)
        body = driver.find_element(By.CSS_SELECTOR, "body")
        for _ in range(scroll_count):
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.5)

        # 4. HTML 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # 5. 블로그 카드에서 제목/링크 추출
        url_soup = (
            soup.select(
                "div > div.sds-comps-vertical-layout.sds-comps-full-layout.N8JknjVu2Kc8aLavF6ZA > a"
            )
            + soup.select(
                "div > div.sds-comps-vertical-layout.sds-comps-full-layout.MkfloTjOr2Rg4LLlLwVA > a"
            )
            +soup.select(
                'div > div.sds-comps-vertical-layout.sds-comps-full-layout._IJWW1BBVoZsf1hqzdtq > a'
                )
        )

        url_soup = url_soup[:max_posts]

        titles = []
        links = []

        for t in url_soup:
            title_text = t.get_text().strip()
            link = t.attrs.get("href", "").strip()
            if not link:
                continue
            print("[BLOG]", title_text, link)
            titles.append(title_text)
            links.append(link)

        # 6. 각 블로그 글 본문 크롤링
        contents = []

        for idx, link in enumerate(links):
            try:
                driver.get(link)

                # 본문/iframe 로딩 대기
                try:
                    wait.until(
                        EC.presence_of_element_located(
                            (
                                By.CSS_SELECTOR,
                                "iframe#mainFrame, div.se-main-container, div#postViewArea",
                            )
                        )
                    )
                except Exception:
                    pass

                # 구버전 블로그(iframe)면 frame 전환
                switched = False
                try:
                    driver.switch_to.frame("mainFrame")
                    switched = True
                except Exception:
                    pass

                text = ""
                # 새 에디터
                try:
                    text = driver.find_element(By.CSS_SELECTOR, "div.se-main-container").text
                except Exception:
                    # 구 에디터
                    try:
                        text = driver.find_element(By.CSS_SELECTOR, "div#postViewArea").text
                    except Exception:
                        text = ""

                contents.append(text)

                if switched:
                    driver.switch_to.default_content()

                print(f"[CONTENT] {idx+1}/{len(links)} 크롤링 완료")

            except Exception as e:
                print(f"[ERROR] {link} 크롤링 중 오류: {e}")
                contents.append("")
                try:
                    driver.switch_to.default_content()
                except Exception:
                    pass

        # 7. 한글만 남긴 컬럼 생성
        only_kor_contents = [extract_korean(c) for c in contents]

        # 8. DataFrame 생성
        review_data = pd.DataFrame(
            {
                "titles": titles,
                "links": links,
                "contents": contents,
                "only_kor_contents": only_kor_contents,
            }
        )

        return review_data

    finally:
        try:
            driver.quit()
        except Exception:
            pass


def build_review_input_text(
    review_series,
    max_reviews: int = 8,
    max_chars_per_review: int = 1500,
) -> str:
    """
    DataFrame의 only_kor_contents 컬럼에서
    최대 max_reviews개, 글당 max_chars_per_review까지 잘라 합친다.
    """
    texts = []
    for raw in review_series[:max_reviews]:
        if not isinstance(raw, str):
            continue
        text = raw.strip()
        if not text:
            continue
        text = text[:max_chars_per_review]
        texts.append(text)

    return "\n\n---\n\n".join(texts)

'''
if __name__ == "__main__":
    # 단일 가게 테스트용
    store = "카피로우 일산밤리단길카페점"
    df = crawl_naver_blog_reviews(store_name=store, scroll_count=1, max_posts=5)
    input_text = build_review_input_text(df["only_kor_contents"])
    print("리뷰 텍스트 길이:", len(input_text))'''


'\nif __name__ == "__main__":\n    # 단일 가게 테스트용\n    store = "카피로우 일산밤리단길카페점"\n    df = crawl_naver_blog_reviews(store_name=store, scroll_count=1, max_posts=5)\n    input_text = build_review_input_text(df["only_kor_contents"])\n    print("리뷰 텍스트 길이:", len(input_text))'

In [ ]:
# llm_summarize.py
from typing import List, Dict

from dotenv import load_dotenv
import os
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

#from crawling import crawl_naver_blog_reviews, build_review_input_text


load_dotenv()  # .env에서 OPENAI_API_KEY 로드


# --- 1) 구조화된 Output 모델 ---
class ReviewExtraction(BaseModel):
    main_menu: List[str] = Field(
        ...,
        description="가게에서 많이 언급되는 대표 메뉴 키워드들 (예: 소금빵, 아메리카노, 고구마라떼)",
    )
    atmosphere: List[str] = Field(
        ...,
        description="가게 분위기, 경험, 매장 특징 키워드들 (예: 아늑한, 감성적인, 좌석이 넓은)",
    )
    recommended_for: List[str] = Field(
        ...,
        description="어떤 유형의 사람이 방문하면 좋은지 (예: 연인과 함께, 친구와 수다, 반려견과 함께)",
    )


# --- 2) LLM + 구조화 출력 준비 ---
base_model = ChatOpenAI(
    model_name="gpt-4o-mini",   # 빠르고 저렴한 모델
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.2,
)

model = base_model.with_structured_output(ReviewExtraction)


# --- 3) 프롬프트 ---
prompt = PromptTemplate.from_template(
    """
너는 한국어 네이버 블로그 리뷰를 분석해서
가게의 대표 메뉴, 분위기, 추천 대상을 키워드로만 뽑는 역할을 한다.

아래 리뷰 텍스트를 보고,
각 항목당 3~4개의 핵심 키워드를 한국어로만 추출해라.

- main_menu: 자주 언급되는 메뉴 이름
- atmosphere: 매장의 분위기/경험/특징
- recommended_for: 어떤 사람이 방문하면 좋을지 (ex. 연인, 친구, 반려견과 함께 등)

반드시 키워드 위주의 짧은 표현만 사용해라.

리뷰 텍스트:
----------------
{text}
----------------
"""
)

# --- 4) LCEL 체인 ---
summarize_chain = prompt | model


# --- 5) 단일 가게용 함수 ---
def extract_review_keywords(input_text: str) -> ReviewExtraction:
    """
    한 가게의 리뷰 전체를 하나의 문자열로 받아서
    대표 메뉴 / 분위기 / 추천 대상을 추출한다.
    """
    if not input_text.strip():
        return ReviewExtraction(main_menu=[], atmosphere=[], recommended_for=[])

    result: ReviewExtraction = summarize_chain.invoke({"text": input_text})
    return result


# --- 6) 여러 가게 batch 처리용 함수 ---
def extract_review_keywords_batch(
    store_to_text: Dict[str, str]
) -> Dict[str, ReviewExtraction]:
    """
    여러 가게의 리뷰 텍스트를 한 번에 LLM에 보내서
    {가게이름: ReviewExtraction} 형태로 결과를 반환한다.
    """
    store_names = list(store_to_text.keys())
    inputs = []

    for store_name in store_names:
        text = store_to_text[store_name] or ""
        text = text.strip()
        inputs.append({"text": text})

    results: List[ReviewExtraction] = summarize_chain.batch(inputs)

    store_to_result: Dict[str, ReviewExtraction] = {}
    for store_name, res in zip(store_names, results):
        if isinstance(res, ReviewExtraction):
            store_to_result[store_name] = res
        else:
            store_to_result[store_name] = ReviewExtraction(
                main_menu=[], atmosphere=[], recommended_for=[]
            )

    return store_to_result


# --- 7) 가게 3개를 한 번에: 크롤링부터 요약까지 ---
if __name__ == "__main__":
    stores = [
        "몽키 일산 밤리단길카페 본점",
        "마제야 마제소바 전문점"
        "뒷북서재",
    ]

    store_to_text: Dict[str, str] = {}

    for store in stores:
        print(f"\n=== {store} 크롤링 시작 ===")
        df = crawl_naver_blog_reviews(store_name=store, scroll_count=1, max_posts=5)
        input_text = build_review_input_text(df["only_kor_contents"])
        print(f"{store} 리뷰 텍스트 길이: {len(input_text)}")
        store_to_text[store] = input_text

    print("\n=== LLM 요약 (3개 가게 batch) 시작 ===")
    batch_result = extract_review_keywords_batch(store_to_text)

    for store, info in batch_result.items():
        print(f"\n######## {store} ########")
        print("대표 메뉴:", info.main_menu)
        print("분위기:", info.atmosphere)
        print("추천 대상:", info.recommended_for)



===  크롤링 시작 ===
 리뷰 텍스트 길이: 0

=== 몽키 일산 밤리단길카페 본점 크롤링 시작 ===
[BLOG] 일산 밤리단길 카페 몽킽｜트리플치즈·모카소금빵·몽키·흑임자슈페너 https://blog.naver.com/diana-0/224034258895
[BLOG] 일산 소금빵 맛집 몽킽 일산밤리단길 카페본점 https://blog.naver.com/sunmi9333/224087769839
[BLOG] 일산 소금빵 맛집 몽킽 밤리단길 카페 본점 데이트 https://blog.naver.com/denimfrog/224039866074
[BLOG] [일산 밤리단길] 소금빵과 몽키슈페너 맛집 몽킽카페_구.베로티오 https://blog.naver.com/saymams/223656434798
[BLOG] 일산 밤리단길카페 몽킽 소금빵과 시그니처 음료 몽키슈페너와 함께 즐기는 커피타임 https://blog.naver.com/chlgmldu1028love/223861112954
[CONTENT] 1/5 크롤링 완료
[CONTENT] 2/5 크롤링 완료
[CONTENT] 3/5 크롤링 완료
[CONTENT] 4/5 크롤링 완료
[CONTENT] 5/5 크롤링 완료
몽키 일산 밤리단길카페 본점 리뷰 텍스트 길이: 7528

=== 마제야 마제소바 전문점뒷북서재 크롤링 시작 ===
[BLOG] 대구 서재세천맛집 한닢 다사세천점 다사 밥집 마제소바 스테이크 덮밥 솔직후기 https://blog.naver.com/smjy74/223358983479
[BLOG] 청라 핫플1. 판코 마제소바 후기 https://blog.naver.com/ggami9997/222824596340
[BLOG] 김포 구래동 백소정 마제소바 돈까스 맛집 https://blog.naver.com/ississ777/223216517180
[BLOG] 대구 다사 맛집 마제소바가 맛있는 한닢라멘 https://blog.naver.com/gjswn61/222657332681
[BLOG] [나고야여행

In [4]:
# parallel_thread_crawl.py
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import Dict

#from crawling import crawl_naver_blog_reviews


def build_review_input_text(review_series, max_reviews=8, max_chars_per_review=1500) -> str:
    texts = []
    for raw in review_series[:max_reviews]:
        if not isinstance(raw, str):
            continue
        text = raw.strip()
        if not text:
            continue
        texts.append(text[:max_chars_per_review])
    return "\n\n---\n\n".join(texts)


def crawl_one_store_to_text(store_name: str) -> tuple[str, str]:
    print(f"[MAIN] {store_name} 크롤링 시작")
    df = crawl_naver_blog_reviews(store_name=store_name, scroll_count=1, max_posts=5)
    if df.empty:
        print(f"[MAIN] {store_name} 결과 없음")
        return store_name, ""
    text = build_review_input_text(df["only_kor_contents"])
    print(f"[MAIN] {store_name} 리뷰 텍스트 길이 = {len(text)}")
    return store_name, text


def crawl_stores_in_threads(stores: list[str], max_workers: int = 3) -> Dict[str, str]:
    store_to_text: Dict[str, str] = {}

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_store = {
            executor.submit(crawl_one_store_to_text, store): store for store in stores
        }

        for future in as_completed(future_to_store):
            store = future_to_store[future]
            try:
                s_name, text = future.result()
                store_to_text[s_name] = text
            except Exception as e:
                print(f"[MAIN] {store} 쓰레드에서 예외 발생: {e}")
                store_to_text[store] = ""

    return store_to_text


if __name__ == "__main__":
    stores = [
        "카피로우 일산밤리단길카페점",
        "몽킽 일산 밤리단길카페 본점",
        "뒷북서재",
    ]

    result = crawl_stores_in_threads(stores, max_workers=3)

    for store, text in result.items():
        print(f"\n===== {store} =====")
        print("텍스트 길이:", len(text))


[MAIN] 카피로우 일산밤리단길카페점 크롤링 시작[MAIN] 몽킽 일산 밤리단길카페 본점 크롤링 시작

[MAIN] 뒷북서재 크롤링 시작
[BLOG] 디저트가 특히 맛있었던 일산 카페, 카피로우 일산밤리단길카페점 방문기 https://blog.naver.com/hidew777/224076747828
[BLOG] 카피로우 일산밤리단길카페점 솔직후기 https://blog.naver.com/rhdmsdl0720/223996105928
[BLOG] 카피로우 일산밤리단길카페점 내돈내산 솔직후기 https://blog.naver.com/rkguscjswo26/223891656310
[BLOG] “카피로우” 일산밤리단길카페점.크렘당쥬?프랑스디저트 천사의크림일산 밤리단길카페추천 https://blog.naver.com/comma2024/223785598110
[BLOG] [경기/고양시] 일산동구 마두동 밤리단길 데이트 카페 추천 무화과 디저트 맛집 찾고 있다면? ‘카피로우 일산밤리단길카페점’ - 아메리카노, 무화과요거트케이크(글루텐프리) https://blog.naver.com/vivid_07/224064134098
[BLOG] 일산 소금빵 맛집 몽킽 일산밤리단길 카페본점 https://blog.naver.com/sunmi9333/224087769839
[BLOG] 일산 소금빵 맛집 몽킽 밤리단길 카페 본점 데이트 https://blog.naver.com/denimfrog/224039866074
[BLOG] 밤리단길카페 추천, 몽킽본점 일산디저트 하면 떠오르는 소금빵맛집 https://blog.naver.com/sojan_euni/224087549948
[BLOG] 일산 밤리단길 카페 소금빵 맛있는 몽킽 다녀옴 https://blog.naver.com/atti_jj0905/224061508313
[BLOG] 밤리단길 카페 추천 일산디저트   몽킽일산밤리단길본점 https://blog.naver.com/22bin_/224096598370
[BLOG] 일산 밤리단길 힐링 북

# 스레드 크롤링(3개) + LLM batch 요약

In [1]:
# crawling.py
import re
import time
from urllib.parse import quote_plus
from typing import Dict, List

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from concurrent.futures import ThreadPoolExecutor, as_completed


def extract_korean(text: str) -> str:
    """문자열에서 한글만 추출해서 공백으로 이어붙임."""
    if not text:
        return ""
    result = re.compile("[가-힣]+").findall(text)
    return " ".join(result)


def build_search_url(store_name: str) -> str:
    """가게 이름으로 네이버 블로그 검색 URL 생성."""
    query = quote_plus(store_name)
    return f"https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query={query}"


def _create_driver() -> webdriver.Chrome:
    """단일 크롤링용 Chrome 드라이버 생성."""
    chrome_options = webdriver.ChromeOptions()
    # 디버깅할 땐 아래 줄 주석 처리하고 실제 창 보면서 하면 좋음
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    driver = webdriver.Chrome(options=chrome_options)
    return driver


def crawl_naver_blog_reviews(
    store_name: str,
    scroll_count: int = 1,
    max_posts: int = 5,
) -> pd.DataFrame:
    """
    네이버 블로그에서 특정 가게 이름으로 검색한 뒤
    블로그 글 제목, 링크, 내용, 한글만 추출한 내용을 크롤링해서 DataFrame 반환.

    컬럼: titles, links, contents, only_kor_contents
    """
    driver = _create_driver()

    try:
        search_url = build_search_url(store_name)
        driver.get(search_url)
        time.sleep(3)

        body = driver.find_element(By.CSS_SELECTOR, "body")
        for _ in range(scroll_count):
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # 클래스 대신 blog.naver.com 도메인 기준으로 링크 찾기
        url_soup = soup.select('a[href*="blog.naver.com"]')

        titles: List[str] = []
        links: List[str] = []

        for t in url_soup[:max_posts]:
            title_text = t.get_text().strip()
            link = t.get("href", "").strip()
            if not link:
                continue

            print(f"[{store_name}] BLOG:", title_text, link)
            titles.append(title_text)
            links.append(link)

        print(f"[INFO] {store_name} 링크 개수: {len(links)}")

        contents: List[str] = []

        for idx, link in enumerate(links):
            try:
                driver.get(link)
                time.sleep(2)

                # 구버전 블로그 iframe 시도
                try:
                    driver.switch_to.frame("mainFrame")
                except Exception:
                    pass

                text = ""
                # 새 에디터
                try:
                    text = driver.find_element(By.CSS_SELECTOR, "div.se-main-container").text
                except Exception:
                    # 구 에디터
                    try:
                        text = driver.find_element(By.CSS_SELECTOR, "div#postViewArea").text
                    except Exception:
                        text = ""

                contents.append(text)
                print(f"[{store_name}] CONTENT {idx+1}/{len(links)} len={len(text)}")

            except Exception as e:
                print(f"[{store_name}] ERROR {link}: {e}")
                contents.append("")
            finally:
                try:
                    driver.switch_to.default_content()
                except Exception:
                    pass

        only_kor_contents = [extract_korean(c) for c in contents]

        return pd.DataFrame(
            {
                "titles": titles,
                "links": links,
                "contents": contents,
                "only_kor_contents": only_kor_contents,
            }
        )

    finally:
        driver.quit()


def build_review_input_text(
    review_series,
    max_reviews: int = 8,
    max_chars_per_review: int = 1500,
) -> str:
    """
    DataFrame의 only_kor_contents에서
    최대 max_reviews개, 글당 max_chars_per_review까지 잘라 합침.
    """
    texts: List[str] = []
    for raw in review_series[:max_reviews]:
        if not isinstance(raw, str):
            continue
        text = raw.strip()
        if not text:
            continue
        texts.append(text[:max_chars_per_review])
    return "\n\n---\n\n".join(texts)


def crawl_one_store_to_text(store_name: str) -> tuple[str, str]:
    """
    스레드에서 실행될 함수.
    가게 하나 크롤링 + 리뷰 합친 텍스트까지 반환.
    """
    print(f"[MAIN] {store_name} 크롤링 시작")
    df = crawl_naver_blog_reviews(store_name=store_name, scroll_count=1, max_posts=5)

    if df.empty:
        print(f"[MAIN] {store_name}: 크롤링 결과 없음")
        return store_name, ""

    text = build_review_input_text(df["only_kor_contents"])
    print(f"[MAIN] {store_name}: 리뷰 텍스트 길이 = {len(text)}")
    return store_name, text


def crawl_stores_in_threads(
    stores: List[str],
    max_workers: int = 3,
) -> Dict[str, str]:
    """
    여러 가게를 스레드로 병렬 크롤링.
    각 가게마다 Chrome 드라이버 하나씩 생성해서 사용.

    return: {가게이름: 리뷰합친텍스트}
    """
    store_to_text: Dict[str, str] = {}

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_store = {
            executor.submit(crawl_one_store_to_text, store): store for store in stores
        }

        for future in as_completed(future_to_store):
            store = future_to_store[future]
            try:
                s_name, text = future.result()
                store_to_text[s_name] = text
            except Exception as e:
                print(f"[MAIN] {store} 쓰레드 예외: {e}")
                store_to_text[store] = ""

    return store_to_text


if __name__ == "__main__":
    # 단독 테스트용
    stores = [
        "카피로우 일산밤리단길카페점",
        "몽키 일산 밤리단길카페 본점",
        "뒷북서재",
    ]

    result = crawl_stores_in_threads(stores, max_workers=3)

    for s, txt in result.items():
        print(f"\n===== {s} =====")
        print("텍스트 길이:", len(txt))


[MAIN] 카피로우 일산밤리단길카페점 크롤링 시작
[MAIN] 몽키 일산 밤리단길카페 본점 크롤링 시작
[MAIN] 뒷북서재 크롤링 시작
[몽키 일산 밤리단길카페 본점] BLOG: 내 블로그 https://blog.naver.com/MyBlog.naver
[몽키 일산 밤리단길카페 본점] BLOG: 블로그 https://section.blog.naver.com/
[몽키 일산 밤리단길카페 본점] BLOG:  https://blog.naver.com/diana-0
[몽키 일산 밤리단길카페 본점] BLOG: 태평한냠뉴 https://blog.naver.com/diana-0
[몽키 일산 밤리단길카페 본점] BLOG: 일산 밤리단길 카페 몽킽｜트리플치즈·모카소금빵·몽키·흑임자슈페너 https://blog.naver.com/diana-0/224034258895
[INFO] 몽키 일산 밤리단길카페 본점 링크 개수: 5
[카피로우 일산밤리단길카페점] BLOG: 내 블로그 https://blog.naver.com/MyBlog.naver
[카피로우 일산밤리단길카페점] BLOG: 블로그 https://section.blog.naver.com/
[카피로우 일산밤리단길카페점] BLOG:  https://blog.naver.com/hidew777
[카피로우 일산밤리단길카페점] BLOG: 행복연구 리뷰메이커의 맛집, 리뷰, 일상 기록 블로그 https://blog.naver.com/hidew777
[카피로우 일산밤리단길카페점] BLOG: 디저트가 특히 맛있었던 일산 카페, 카피로우 일산밤리단길카페점 방문기 https://blog.naver.com/hidew777/224076747828
[INFO] 카피로우 일산밤리단길카페점 링크 개수: 5
[몽키 일산 밤리단길카페 본점] CONTENT 1/5 len=0
[뒷북서재] BLOG: 내 블로그 https://blog.naver.com/MyBlog.naver
[뒷북서재] BLOG: 블로그 https://section.blog.naver.com/


In [ ]:
# llm_summarize.py
from typing import List, Dict

import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

from crawling import crawl_stores_in_threads


load_dotenv()  # .env에서 OPENAI_API_KEY 로드


# --- 1) 구조화된 Output 모델 ---
class ReviewExtraction(BaseModel):
    main_menu: List[str] = Field(
        ...,
        description="가게에서 많이 언급되는 대표 메뉴 키워드들 (예: 소금빵, 아메리카노, 고구마라떼)",
    )
    atmosphere: List[str] = Field(
        ...,
        description="가게 분위기, 경험, 매장 특징 키워드들 (예: 아늑한, 감성적인, 좌석이 넓은)",
    )
    recommended_for: List[str] = Field(
        ...,
        description="어떤 유형의 사람이 방문하면 좋은지 (예: 연인과 함께, 친구와 수다, 반려견과 함께)",
    )


# --- 2) LLM + 구조화 출력 준비 ---
base_model = ChatOpenAI(
    model_name="gpt-4o-mini",        # 빠르고 저렴한 모델
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.2,
)

model = base_model.with_structured_output(ReviewExtraction)


# --- 3) 프롬프트 ---
prompt = PromptTemplate.from_template(
    """
너는 한국어 네이버 블로그 리뷰를 분석해서
가게의 대표 메뉴, 분위기, 추천 대상을 키워드로만 뽑는 역할을 한다.

아래 리뷰 텍스트를 보고,
각 항목당 3~4개의 핵심 키워드를 한국어로만 추출해라.

- main_menu: 자주 언급되는 메뉴 이름
- atmosphere: 매장의 분위기/경험/특징
- recommended_for: 어떤 사람이 방문하면 좋을지 (ex. 연인, 친구, 반려견과 함께 등)

반드시 키워드 위주의 짧은 표현만 사용해라.

리뷰 텍스트:
----------------
{text}
----------------
"""
)

# --- 4) LCEL 체인 ---
summarize_chain = prompt | model


# --- 5) 단일 가게용 함수 ---
def extract_review_keywords(input_text: str) -> ReviewExtraction:
    if not input_text.strip():
        return ReviewExtraction(main_menu=[], atmosphere=[], recommended_for=[])
    result: ReviewExtraction = summarize_chain.invoke({"text": input_text})
    return result


# --- 6) 여러 가게 batch 처리 ---
def extract_review_keywords_batch(
    store_to_text: Dict[str, str]
) -> Dict[str, ReviewExtraction]:
    store_names = list(store_to_text.keys())
    inputs = [{"text": store_to_text[name]} for name in store_names]

    results: List[ReviewExtraction] = summarize_chain.batch(inputs)

    store_to_result: Dict[str, ReviewExtraction] = {}
    for name, res in zip(store_names, results):
        if isinstance(res, ReviewExtraction):
            store_to_result[name] = res
        else:
            store_to_result[name] = ReviewExtraction(
                main_menu=[], atmosphere=[], recommended_for=[]
            )

    return store_to_result


# --- 7) 전체 파이프라인: 3개 가게 크롤링 + 요약 ---
if __name__ == "__main__":
    stores = [
        "카피로우 일산밤리단길카페점",
        "몽키 일산 밤리단길카페 본점",
        "뒷북서재",
    ]

    # 1) 병렬 크롤링 (가게당 Chrome 하나, ThreadPool)
    store_to_text = crawl_stores_in_threads(stores, max_workers=3)

    # 2) LLM batch 요약
    print("\n=== LLM 요약 (batch) 시작 ===")
    batch_result = extract_review_keywords_batch(store_to_text)

    # 3) 결과 출력
    for store, info in batch_result.items():
        print(f"\n######## {store} ########")
        print("대표 메뉴:", info.main_menu)
        print("분위기:", info.atmosphere)
        print("추천 대상:", info.recommended_for)


[MAIN] 카피로우 일산밤리단길카페점 크롤링 시작
[MAIN] 몽키 일산 밤리단길카페 본점 크롤링 시작
[MAIN] 어쩌구 카페 홍대점 크롤링 시작
[어쩌구 카페 홍대점] BLOG: 내 블로그 https://blog.naver.com/MyBlog.naver
[어쩌구 카페 홍대점] BLOG: 블로그 https://section.blog.naver.com/
[어쩌구 카페 홍대점] BLOG:  https://blog.naver.com/my_day-to-day
[어쩌구 카페 홍대점] BLOG: 덕질하는 날들 https://blog.naver.com/my_day-to-day
[어쩌구 카페 홍대점] BLOG: 실바니안 아기 셰프의 다락방 카페 엔제리너스 L7홍대점 페르시안 고양이 데려오기! https://blog.naver.com/my_day-to-day/223969460076
[INFO] 어쩌구 카페 홍대점 링크 개수: 5
[카피로우 일산밤리단길카페점] BLOG: 내 블로그 https://blog.naver.com/MyBlog.naver
[카피로우 일산밤리단길카페점] BLOG: 블로그 https://section.blog.naver.com/
[카피로우 일산밤리단길카페점] BLOG:  https://blog.naver.com/hidew777
[카피로우 일산밤리단길카페점] BLOG: 행복연구 리뷰메이커의 맛집, 리뷰, 일상 기록 블로그 https://blog.naver.com/hidew777
[카피로우 일산밤리단길카페점] BLOG: 디저트가 특히 맛있었던 일산 카페, 카피로우 일산밤리단길카페점 방문기 https://blog.naver.com/hidew777/224076747828
[INFO] 카피로우 일산밤리단길카페점 링크 개수: 5
[몽키 일산 밤리단길카페 본점] BLOG: 내 블로그 https://blog.naver.com/MyBlog.naver
[몽키 일산 밤리단길카페 본점] BLOG: 블로그 https://section.blog.naver.com/
[몽키 일산 밤